In [5]:
from apyori import apriori
import pandas as pd
import os

In [19]:
import Orange


ModuleNotFoundError: No module named 'Orange'

In [3]:
merged_data = pd.read_csv('out.csv')
colonnes_inutiles = ['num_veh','Num_Acc',
                                'locp',
                                'actp',
                                'etatp',
                                'catv',
                                'hrmn',
                                'mois',
                                'jour',
                                'catvopp',
                                'occutc',
                                'place']

merged_data = merged_data.drop(colonnes_inutiles,axis = 1)


y = merged_data.grav
merged_data_sans_grav = merged_data.drop(['grav'],axis = 1)

In [4]:
def mise_en_place_apriori(data_accident, min_sup=0.40, min_conf=0.90):
    
    listing_apriori_bis = []

    for indexe in data_accident.index:
        event = []
        for colonne in data_accident.columns:
            if not (data_accident[colonne][indexe] == 0):
                event.append(colonne + str(data_accident[colonne][indexe]))
        listing_apriori_bis.append(event)

    results_apriori_decedes = list(apriori(listing_apriori_bis, min_support = min_sup, min_confidence = min_conf))

    tableau_apriori = []

    for x in range(0,len(results_apriori_decedes)):
        event = [list(results_apriori_decedes[x].items), results_apriori_decedes[x].support, results_apriori_decedes[x].ordered_statistics[0].confidence]
        tableau_apriori.append(event)

    tableau_apriori = pd.DataFrame(tableau_apriori, columns=['transaction','support','confiance'])
    os.system('say "fini"')
    return tableau_apriori


In [6]:
apriori_sur_tout = mise_en_place_apriori(merged_data_sans_grav)
apriori_sur_morts = mise_en_place_apriori(merged_data[merged_data.grav == 2].drop('grav', axis=1))
apriori_sur_indemmes = mise_en_place_apriori(merged_data[merged_data.grav == 1].drop('grav', axis=1))

In [7]:
apriori_sur_tout

,transaction,support,confiance
0,"[secu11.0, catv_gen2]",0.601629,0.928051
1,"[secu11.0, agg1, catv_gen2]",0.416047,0.920300
2,"[secu11.0, atm1, catv_gen2]",0.462227,0.925910
3,"[secu11.0, catu1, catv_gen2]",0.447142,0.921195
4,"[secu11.0, catvopp_gen2, catv_gen2]",0.448468,0.947163
5,"[secu11.0, int1, catv_gen2]",0.482185,0.925021
6,"[secu11.0, lum1, catv_gen2]",0.415492,0.923103


In [8]:
def intersection_2_listes(L1, L2):
    """List[Int] * List[Int] -> List[Int]
    Retourne l'intersection des deux listes L1 et L2."""
 
    result = []
 
    for n in L1:
        for x in L2:
            if n == x and n not in result:
                result.append(n)
    result.sort()
 
    return result

In [9]:
association_tout = list(apriori_sur_tout.transaction)
association_indemmes = list(apriori_sur_indemmes.transaction)
association_morts = list(apriori_sur_morts.transaction)

tout_inter_morts = intersection_2_listes(association_tout, association_morts)
tout_inter_indemmes = intersection_2_listes(association_tout,association_indemmes)
indemmes_inter_morts = intersection_2_listes(association_indemmes,association_morts)

In [10]:
tout_inter_morts

[['secu11.0', 'catv_gen2']]

In [11]:
tout_inter_indemmes

[['secu11.0', 'agg1', 'catv_gen2'], ['secu11.0', 'catu1', 'catv_gen2']]

In [12]:
indemmes_inter_morts

[]

In [13]:
apriori_sur_morts

,transaction,support,confiance
0,"[agg1, int1]",0.728389,0.911068
1,"[int1, catv_gen2]",0.631714,0.910096
2,"[secu11.0, catv_gen2]",0.433248,0.964692
3,"[secu11.0, int1]",0.411253,0.915718
4,"[agg1, int1, atm1]",0.544757,0.907155
5,"[agg1, catu1, int1]",0.545780,0.905772
6,"[agg1, int1, catv_gen2]",0.532481,0.922872
7,"[agg1, catvopp_gen2, int1]",0.543223,0.906143
8,"[sexe1, agg1, int1]",0.541176,0.909716
9,"[int1, atm1, catv_gen2]",0.441432,0.905561
